In [8]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [9]:
#APRTC BUS DETAILS

In [10]:

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 6):  # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            driver.quit()

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")
            driver.quit()
            
# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('ap_bus_details.csv', index=False)


Error occurred while accessing page 2: Message: session not created
from chrome not reachable
Stacktrace:
	GetHandleVerifier [0x00007FF652599412+29090]
	(No symbol) [0x00007FF65250E239]
	(No symbol) [0x00007FF6523CB009]
	(No symbol) [0x00007FF6523B76D4]
	(No symbol) [0x00007FF65240C558]
	(No symbol) [0x00007FF6524072AC]
	(No symbol) [0x00007FF652401E1A]
	(No symbol) [0x00007FF65245233D]
	(No symbol) [0x00007FF6524519F0]
	(No symbol) [0x00007FF652446493]
	(No symbol) [0x00007FF6524109D1]
	(No symbol) [0x00007FF652411B31]
	GetHandleVerifier [0x00007FF6528B871D+3302573]
	GetHandleVerifier [0x00007FF652904243+3612627]
	GetHandleVerifier [0x00007FF6528FA417+3572135]
	GetHandleVerifier [0x00007FF652655EB6+801862]
	(No symbol) [0x00007FF65251945F]
	(No symbol) [0x00007FF652514FB4]
	(No symbol) [0x00007FF652515140]
	(No symbol) [0x00007FF65250461F]
	BaseThreadInitThunk [0x00007FFA416D7614+20]
	RtlUserThreadStart [0x00007FFA431826B1+33]



In [11]:
df

                   Route_Name  \
0     Hyderabad to Vijayawada   
1     Hyderabad to Vijayawada   
2     Hyderabad to Vijayawada   
3     Hyderabad to Vijayawada   
4     Hyderabad to Vijayawada   
...                       ...   
1325     Hyderabad to Nandyal   
1326     Hyderabad to Nandyal   
1327     Hyderabad to Nandyal   
1328     Hyderabad to Nandyal   
1329     Hyderabad to Nandyal   

                                             Route_Link         Bus_Name  \
0     https://www.redbus.in/bus-tickets/hyderabad-to...  APSRTC - 351880   
1     https://www.redbus.in/bus-tickets/hyderabad-to...    APSRTC - 3563   
2     https://www.redbus.in/bus-tickets/hyderabad-to...   APSRTC - 35266   
3     https://www.redbus.in/bus-tickets/hyderabad-to...                    
4     https://www.redbus.in/bus-tickets/hyderabad-to...                    
...                                                 ...              ...   
1325  https://www.redbus.in/bus-tickets/hyderabad-to...                

In [12]:
#ASSAM BUS DETAILS

In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/astc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 6):  # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            driver.quit()


        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")
            driver.quit()

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('assam_bus_details.csv', index=False)


Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/sibsagar-to-north-lakhimpur: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF652599412+29090]
	(No symbol) [0x00007FF65250E239]
	(No symbol) [0x00007FF6523CB1DA]
	(No symbol) [0x00007FF65241EFE7]
	(No symbol) [0x00007FF65241F23C]
	(No symbol) [0x00007FF6524697C7]
	(No symbol) [0x00007FF65244672F]
	(No symbol) [0x00007FF6524665A2]
	(No symbol) [0x00007FF652446493]
	(No symbol) [0x00007FF6524109D1]
	(No symbol) [0x00007FF652411B31]
	GetHandleVerifier [0x00007FF6528B871D+3302573]
	GetHandleVerifier [0x00007FF652904243+3612627]
	GetHandleVerifier [0x00007FF6528FA417+3572135]
	GetHandleVerifier [0x00007FF652655EB6+801862]
	(No symbol) [0x00007FF65251945F]
	(No symbol) [0x00007FF652514FB4]
	(No symbol) [0x00007FF652515140]
	(No symbol) [0x00007FF65250461F]
	BaseThreadInitThunk [0x00007FFA416D7614+20]
	RtlUserThreadStart [0x00007FFA431826B1+33]

Error occurred while scraping bus details for https://www.redbus.i

In [14]:
df

                         Route_Name  \
0                Tezpur to Guwahati   
1                Tezpur to Guwahati   
2                Tezpur to Guwahati   
3                Tezpur to Guwahati   
4                Tezpur to Guwahati   
..                              ...   
477            Guwahati to Golaghat   
478            Guwahati to Golaghat   
479              Jorhat to Gogamukh   
480  Dibrugarh to Biswanath Charali   
481     Golaghat to North Lakhimpur   

                                            Route_Link  \
0    https://www.redbus.in/bus-tickets/tezpur-to-gu...   
1    https://www.redbus.in/bus-tickets/tezpur-to-gu...   
2    https://www.redbus.in/bus-tickets/tezpur-to-gu...   
3    https://www.redbus.in/bus-tickets/tezpur-to-gu...   
4    https://www.redbus.in/bus-tickets/tezpur-to-gu...   
..                                                 ...   
477  https://www.redbus.in/bus-tickets/guwahati-to-...   
478  https://www.redbus.in/bus-tickets/guwahati-to-...   
479  http

In [15]:
#chandigar bus details

In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 6):  # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            driver.quit()

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")
            driver.quit()
# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('chandigarh_bus_details.csv', index=False)

In [17]:
df

                      Route_Name  \
0     Yamuna Nagar to Chandigarh   
1     Yamuna Nagar to Chandigarh   
2     Yamuna Nagar to Chandigarh   
3     Yamuna Nagar to Chandigarh   
4     Yamuna Nagar to Chandigarh   
...                          ...   
1092      Chandigarh to Firozpur   
1093      Chandigarh to Firozpur   
1094      Chandigarh to Firozpur   
1095      Chandigarh to Firozpur   
1096      Chandigarh to Firozpur   

                                             Route_Link  \
0     https://www.redbus.in/bus-tickets/yamuna-nagar...   
1     https://www.redbus.in/bus-tickets/yamuna-nagar...   
2     https://www.redbus.in/bus-tickets/yamuna-nagar...   
3     https://www.redbus.in/bus-tickets/yamuna-nagar...   
4     https://www.redbus.in/bus-tickets/yamuna-nagar...   
...                                                 ...   
1092  https://www.redbus.in/bus-tickets/chandigarh-t...   
1093  https://www.redbus.in/bus-tickets/chandigarh-t...   
1094  https://www.redbus.in/bus-tick

In [18]:
#Himachal Pradesh bus details

In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 6):  # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            driver.quit()

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")
            driver.quit()

            
# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('himachal_bus_details.csv', index=False)

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/delhi-to-baddi-himachal-pradesh: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF754DD9412+29090]
	(No symbol) [0x00007FF754D4E239]
	(No symbol) [0x00007FF754C0B1DA]
	(No symbol) [0x00007FF754C5EFE7]
	(No symbol) [0x00007FF754C5F23C]
	(No symbol) [0x00007FF754CA97C7]
	(No symbol) [0x00007FF754C8672F]
	(No symbol) [0x00007FF754CA65A2]
	(No symbol) [0x00007FF754C86493]
	(No symbol) [0x00007FF754C509D1]
	(No symbol) [0x00007FF754C51B31]
	GetHandleVerifier [0x00007FF7550F871D+3302573]
	GetHandleVerifier [0x00007FF755144243+3612627]
	GetHandleVerifier [0x00007FF75513A417+3572135]
	GetHandleVerifier [0x00007FF754E95EB6+801862]
	(No symbol) [0x00007FF754D5945F]
	(No symbol) [0x00007FF754D54FB4]
	(No symbol) [0x00007FF754D55140]
	(No symbol) [0x00007FF754D4461F]
	BaseThreadInitThunk [0x00007FFA416D7614+20]
	RtlUserThreadStart [0x00007FFA431826B1+33]

Error occurred while accessing page 5: Message: 
Stacktrace:
	

In [21]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 69,Himsuta AC Seater Volvo/Scania 2+2,00:40,08h 50m,09:30,4.7,INR 912,33 Seats available
1,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 6,Himsuta AC Seater Volvo/Scania 2+2,06:45,09h 25m,16:10,4.4,INR 912,28 Seats available
2,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 592,A/C Executive (2+3),08:05,10h 05m,18:10,2.3,INR 632,42 Seats available
3,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 129,Ordinary 3+2 Non AC Seater,08:50,09h 50m,18:40,2.6,INR 512,37 Seats available
4,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 7,Himsuta AC Seater Volvo/Scania 2+2,09:25,09h 05m,18:30,4.5,INR 912,31 Seats available
...,...,...,...,...,...,...,...,...,...,...
1010,Manali to Delhi,https://www.redbus.in/bus-tickets/manali-to-delhi,Reo India Travels,Volvo A/C B11R Multi Axle Semi Sleeper (2+2),19:10,10h 55m,06:05,3.8,567,34 Seats available
1011,Manali to Delhi,https://www.redbus.in/bus-tickets/manali-to-delhi,Deltin Travels,Scania Multi-Axle AC Semi Sleeper (2+2),19:30,10h 30m,06:00,2.9,INR 700,38 Seats available
1012,Manali to Delhi,https://www.redbus.in/bus-tickets/manali-to-delhi,Reo India Travels,Volvo A/C Semi Sleeper (2+2),19:35,12h 00m,07:35,0,567,17 Seats available
1013,Manali to Delhi,https://www.redbus.in/bus-tickets/manali-to-delhi,Himalayan Nomad Ktc,Volvo A/C Semi Sleeper (2+2),16:30,11h 30m,04:00,0,744,41 Seats available


In [22]:
#KAAC bus details

In [23]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/kaac-transport"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 3):  # There are 2 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            driver.quit()       

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")
            driver.quit()
# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('kaac_bus_details.csv', index=False)

In [24]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Diphu to Guwahati,https://www.redbus.in/bus-tickets/diphu-to-guw...,KAAC TRANSPORT - 157249,Non AC Seater 2+2,04:00,05h 30m,09:30,4.2,361,21 Seats available
1,Diphu to Guwahati,https://www.redbus.in/bus-tickets/diphu-to-guw...,Kareng Travels,Non AC Seater (2+2),04:30,05h 00m,09:30,4.1,361,5 Seats available
2,Diphu to Guwahati,https://www.redbus.in/bus-tickets/diphu-to-guw...,Kareng Travels,Bharat Benz A/C Seater (2+1),10:00,06h 00m,16:00,4.4,INR 400,9 Seats available
3,Diphu to Guwahati,https://www.redbus.in/bus-tickets/diphu-to-guw...,Kareng Travels,Non AC Seater (2+1),11:30,05h 15m,16:45,4.0,INR 380,29 Seats available
4,Guwahati to Diphu,https://www.redbus.in/bus-tickets/guwahati-to-...,KAAC TRANSPORT - 157250,Non AC Seater 2+2,11:30,06h 15m,17:45,4.5,361,40 Seats available
5,Guwahati to Diphu,https://www.redbus.in/bus-tickets/guwahati-to-...,Kareng Travels,Non AC Seater (2+1),05:00,06h 00m,11:00,4.0,INR 380,26 Seats available
6,Guwahati to Diphu,https://www.redbus.in/bus-tickets/guwahati-to-...,Kareng Travels,A/C Seater (2+1),09:30,06h 45m,16:15,4.3,INR 400,25 Seats available
7,Guwahati to Diphu,https://www.redbus.in/bus-tickets/guwahati-to-...,Kareng Travels,Non AC Seater (2+2),11:45,06h 45m,18:30,4.5,361,28 Seats available
8,Dokmoka to Guwahati,https://www.redbus.in/bus-tickets/dokmoka-to-g...,KAAC TRANSPORT - 157249,Non AC Seater 2+2,05:45,03h 45m,09:30,4.5,INR 250,38 Seats available
9,Dokmoka to Guwahati,https://www.redbus.in/bus-tickets/dokmoka-to-g...,Kareng Travels,Non AC Seater (2+2),06:00,03h 30m,09:30,3.4,266,5 Seats available


In [25]:
#kerala bus details

In [26]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 3):  # There are 2 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            driver.quit()

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")
            driver.quit()
            
# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('kerala_bus_details.csv', index=False)

In [27]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 52,SWIFT-GARUDA A/C SEATER BUS,12:01,07h 54m,19:55,3.5,INR 945,19 Seats available
1,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 1378,AC MULTI AXLE,13:46,07h 39m,21:25,2.0,INR 1099,35 Seats available
2,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 129,SWIFT-GARUDA A/C SEATER BUS,14:15,08h 35m,22:50,4.2,INR 945,39 Seats available
3,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 235,Swift Deluxe Non AC Air Bus (2+2),15:00,09h 01m,00:01,3.9,INR 801,24 Seats available
4,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 370,AC MULTI AXLE,15:25,06h 55m,22:20,2.9,INR 1099,26 Seats available
...,...,...,...,...,...,...,...,...,...,...
816,Kottayam to Kozhikode,https://www.redbus.in/bus-tickets/kottayam-to-...,A1 Travels,A/C Seater / Sleeper (2+1),23:59,07h 01m,07:00,3.2,INR 700,24 Seats available
817,Kottayam to Kozhikode,https://www.redbus.in/bus-tickets/kottayam-to-...,EMERALD TRAVELS,A/C Sleeper (2+1),23:00,05h 30m,04:30,2.9,INR 1100,13 Seats available
818,Kottayam to Kozhikode,https://www.redbus.in/bus-tickets/kottayam-to-...,Jacobs Travels,A/C Semi Sleeper (2+2),18:30,08h 00m,02:30,2.7,INR 950,34 Seats available
819,Kottayam to Kozhikode,https://www.redbus.in/bus-tickets/kottayam-to-...,AURA CONNECT,A/C Sleeper (2+1),18:15,08h 15m,02:30,2.7,3899,30 Seats available


In [28]:
#Rajasthan bus details

In [29]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 4):  # There are 3 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            driver.quit()  # Close the driver after scraping each page

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")
            driver.quit()  # Ensure driver is closed even if an error occurs

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('rajasthan_bus_details.csv', index=False)

Error occurred while accessing page 3: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF754DD9412+29090]
	(No symbol) [0x00007FF754D4E239]
	(No symbol) [0x00007FF754C0B1DA]
	(No symbol) [0x00007FF754C5EFE7]
	(No symbol) [0x00007FF754C5F23C]
	(No symbol) [0x00007FF754CA97C7]
	(No symbol) [0x00007FF754C8672F]
	(No symbol) [0x00007FF754CA65A2]
	(No symbol) [0x00007FF754C86493]
	(No symbol) [0x00007FF754C509D1]
	(No symbol) [0x00007FF754C51B31]
	GetHandleVerifier [0x00007FF7550F871D+3302573]
	GetHandleVerifier [0x00007FF755144243+3612627]
	GetHandleVerifier [0x00007FF75513A417+3572135]
	GetHandleVerifier [0x00007FF754E95EB6+801862]
	(No symbol) [0x00007FF754D5945F]
	(No symbol) [0x00007FF754D54FB4]
	(No symbol) [0x00007FF754D55140]
	(No symbol) [0x00007FF754D4461F]
	BaseThreadInitThunk [0x00007FFA416D7614+20]
	RtlUserThreadStart [0x00007FFA431826B1+33]



In [30]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,RSRTC - 174534,Express Non AC Seater 2+3,05:45,06h 15m,12:00,4.2,INR 299,41 Seats available
1,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,RSRTC - 158798,Super Luxury Volvo AC Seater Pushback 2+2,06:01,04h 59m,11:00,4.3,INR 602,19 Seats available
2,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,RSRTC - 146395,Express Non AC Seater 2+3,06:45,07h 15m,14:00,4.2,INR 299,42 Seats available
3,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,RSRTC - 194255,Express Non AC Seater 2+3,07:45,06h 35m,14:20,4.2,INR 299,46 Seats available
4,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,RSRTC - 174533,Express Non AC Seater 2+3,08:45,07h 15m,16:00,4.2,INR 299,42 Seats available
...,...,...,...,...,...,...,...,...,...,...
1068,Jaipur (Rajasthan) to Kota(Rajasthan),https://www.redbus.in/bus-tickets/jaipur-to-ko...,Shree Balaji travels SBG,A/C Seater (2+3),07:30,05h 25m,12:55,2.9,INR 400,55 Seats available
1069,Jaipur (Rajasthan) to Kota(Rajasthan),https://www.redbus.in/bus-tickets/jaipur-to-ko...,Priya Bus Service,AC Seater (2+2),16:00,05h 00m,21:00,1.6,INR 399,42 Seats available
1070,Jaipur (Rajasthan) to Kota(Rajasthan),https://www.redbus.in/bus-tickets/jaipur-to-ko...,Mahakal Laxmi Travels,A/C Seater (2+3),12:25,05h 40m,18:05,1.2,INR 304.76,49 Seats available
1071,Jaipur (Rajasthan) to Kota(Rajasthan),https://www.redbus.in/bus-tickets/jaipur-to-ko...,Ganesh Travels,NON A/C Seater/ Sleeper (2+1),22:00,06h 00m,04:00,1.7,INR 350,37 Seats available


In [31]:
#SouthBengal bus details

In [32]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
            
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 6):  # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            driver.quit()

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")
            driver.quit()
            
# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('sb_bus_details.csv', index=False)

In [33]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-DURGAPUR - KOLKATA - 05:00 (DGP DEPOT) -...,Non AC Seater (2+3),05:00,03h 30m,08:30,4.3,INR 150,42 Seats available
1,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-DURGAPUR - KARUNAMOYEE - 05:15 (DGP DEPO...,Non AC Seater (2+3),05:15,04h 00m,09:15,4.2,INR 155,10 Seats available
2,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-DURGAPUR - KOLKATA - 05:30 (DGP DEPOT) -...,Non AC Seater (2+3),05:30,03h 30m,09:00,4.0,INR 150,51 Seats available
3,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-DURGAPUR - KOLKATA - L/S - 06:00 (DGP DE...,Non AC Seater (2+3),06:00,03h 30m,09:30,3.0,INR 150,42 Seats available
4,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-DURGAPUR - KARUNAMOYEE - 06:15 (DGP DEPO...,Non AC Seater (2+3),06:15,04h 00m,10:15,3.9,INR 155,22 Seats available
...,...,...,...,...,...,...,...,...,...,...
821,Barasat (West Bengal) to Digha,https://www.redbus.in/bus-tickets/barasat-west...,Ankita Paribahan (Satya),A/C Seater Push Back (2+3),05:40,05h 55m,11:35,2.7,INR 320,21 Seats available
822,Barasat (West Bengal) to Digha,https://www.redbus.in/bus-tickets/barasat-west...,Snemita Parisheba,AC Seater (2+3),05:50,04h 30m,10:20,1.0,399,21 Seats available
823,Barasat (West Bengal) to Digha,https://www.redbus.in/bus-tickets/barasat-west...,Satya Paribahan,A/C Executive (2+3),06:00,04h 00m,10:00,1.7,INR 320,61 Seats available
824,Barasat (West Bengal) to Digha,https://www.redbus.in/bus-tickets/barasat-west...,Aradhana Travels,A/C Seater / Sleeper (2+2),02:30,05h 30m,08:00,3.4,555,59 Seats available


In [34]:
#Telungana Bus details

In [35]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 4):  # There are 3 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            driver.quit()


        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")
            driver.quit()
# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('Telangana_bus_details.csv', index=False)

In [36]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 351880,VENNELA (A.C. SLEEPER),00:40,05h 25m,06:05,4.7,INR 737,12 Seats available
1,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3563,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",01:00,06h 15m,07:15,3.8,INR 412,32 Seats available
2,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 35266,VENNELA (A.C. SLEEPER),01:25,05h 24m,06:49,3.9,INR 737,27 Seats available
3,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3590,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",01:45,06h 15m,08:00,4.5,INR 412,33 Seats available
4,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3058,VENNELA (A.C. SLEEPER),02:20,05h 20m,07:40,2.9,INR 781,21 Seats available
...,...,...,...,...,...,...,...,...,...,...
1520,Hyderabad to Addanki,https://www.redbus.in/bus-tickets/hyderabad-to...,Sri Krishna Travels,NON A/C Semi Sleeper / Sleeper (2+1),20:00,08h 30m,04:30,2.7,INR 550,34 Seats available
1521,Hyderabad to Addanki,https://www.redbus.in/bus-tickets/hyderabad-to...,Vihari Travels,NON A/C Seater/ Sleeper (2+1),20:00,07h 45m,03:45,4.6,474,27 Seats available
1522,Hyderabad to Addanki,https://www.redbus.in/bus-tickets/hyderabad-to...,VV Vinayak Travels,A/C Seater / Sleeper (2+1),22:10,06h 50m,05:00,4.3,INR 590,35 Seats available
1523,Hyderabad to Addanki,https://www.redbus.in/bus-tickets/hyderabad-to...,Jagan Travels,Scania AC Multi Axle Sleeper (2+1),20:40,05h 50m,02:30,0,INR 935,29 Seats available


In [37]:
#Uttar Pradesh UPRTC bus details

In [38]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 6):  # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            driver.quit()

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")
            driver.quit()
# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('up_bus_details.csv', index=False)

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/agra-to-lucknow: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=128.0.6613.138); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF754DD9412+29090]
	(No symbol) [0x00007FF754D4E239]
	(No symbol) [0x00007FF754C0B1DA]
	(No symbol) [0x00007FF754C11CE3]
	(No symbol) [0x00007FF754C14081]
	(No symbol) [0x00007FF754C14120]
	(No symbol) [0x00007FF754C58C9B]
	(No symbol) [0x00007FF754C866EA]
	(No symbol) [0x00007FF754C526C6]
	(No symbol) [0x00007FF754C86900]
	(No symbol) [0x00007FF754CA65A2]
	(No symbol) [0x00007FF754C86493]
	(No symbol) [0x00007FF754C509D1]
	(No symbol) [0x00007FF754C51B31]
	GetHandleVerifier [0x00007FF7550F871D+3302573]
	GetHandleVerifier [0x00007FF755144243+3612627]
	GetHandleVerifier [0x00007FF75513

In [39]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...,UPSRTC - RKD0134,Janrath AC Seater 2+3,00:31,05h 00m,05:31,1.9,INR 520,48 Seats available
1,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...,UPSRTC - BLY0112,Janrath AC Seater 2+2,05:31,04h 44m,10:15,2.3,INR 598,36 Seats available
2,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...,UPSRTC - GRH0229,Ordinary Non AC Seater 2+3,06:00,06h 30m,12:30,3.3,INR 418,52 Seats available
3,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...,UPSRTC - STD0161,Ordinary Non AC Seater 2+3,06:00,07h 16m,13:16,3.3,INR 448,52 Seats available
4,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...,UPSRTC - RKD0129,Janrath AC Seater 2+2,06:01,05h 00m,11:01,3.1,INR 598,36 Seats available
...,...,...,...,...,...,...,...,...,...,...
2404,Etah (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/etah-up-to-d...,UPSRTC - MNP0132,Ordinary Non AC Seater 2+3,23:32,04h 40m,04:12,0,INR 371,52 Seats available
2405,Etah (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/etah-up-to-d...,Chaturvedi Bus Service,NON AC Seater / Sleeper 2+1,20:30,06h 00m,02:30,0,INR 620,45 Seats available
2406,Etah (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/etah-up-to-d...,Chaturvedi Bus Service,NON A/C Seater/ Sleeper (2+1),21:00,06h 30m,03:30,0,INR 570,51 Seats available
2407,Etah (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/etah-up-to-d...,Chaturvedi Bus Service,NON A/C Seater/ Sleeper (2+1),22:45,06h 30m,05:15,0,INR 600,51 Seats available


In [40]:
#West Bengal Bus details

In [41]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 6):  # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            driver.quit() 
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")
            driver.quit()
# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('wb_bus_details.csv', index=False)

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/kolkata-to-mayapur-iskcon: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF754DD9412+29090]
	(No symbol) [0x00007FF754D4E239]
	(No symbol) [0x00007FF754C0B1DA]
	(No symbol) [0x00007FF754C5EFE7]
	(No symbol) [0x00007FF754C5F23C]
	(No symbol) [0x00007FF754CA97C7]
	(No symbol) [0x00007FF754C8672F]
	(No symbol) [0x00007FF754CA65A2]
	(No symbol) [0x00007FF754C86493]
	(No symbol) [0x00007FF754C509D1]
	(No symbol) [0x00007FF754C51B31]
	GetHandleVerifier [0x00007FF7550F871D+3302573]
	GetHandleVerifier [0x00007FF755144243+3612627]
	GetHandleVerifier [0x00007FF75513A417+3572135]
	GetHandleVerifier [0x00007FF754E95EB6+801862]
	(No symbol) [0x00007FF754D5945F]
	(No symbol) [0x00007FF754D54FB4]
	(No symbol) [0x00007FF754D55140]
	(No symbol) [0x00007FF754D4461F]
	BaseThreadInitThunk [0x00007FFA416D7614+20]
	RtlUserThreadStart [0x00007FFA431826B1+33]

Error occurred while accessing page 5: Message: 
Stacktrace:
	GetHan

In [42]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,WBTC (CTC) HABRA-DIGHA via Esplanade - 176|04:00,Non AC Seater (2+3),04:00,05h 50m,09:50,2.1,INR 165,41 Seats available
1,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,WBTC (CTC) BARASAT-DIGHA via Bally - 38|05:40,Non AC Seater (2+3),05:40,04h 30m,10:10,3.1,INR 161,34 Seats available
2,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,WBTC (CTC) HABRA-DIGHA via Bally - 26|10:00,Non AC Seater (2+3),10:00,04h 45m,14:45,3.3,INR 161,30 Seats available
3,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,"WBTC (CTC) BARASAT-DIGHA via Esplanade, Karuna...",Non AC Seater (2+3),10:15,05h 20m,15:35,3.0,INR 165,45 Seats available
4,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,WBTC (CTC) HABRA-DIGHA via Bally - 26|10:40,Non AC Seater (2+3),10:40,04h 45m,15:25,3.6,INR 161,41 Seats available
...,...,...,...,...,...,...,...,...,...,...
644,Habra to Heria,https://www.redbus.in/bus-tickets/habra-to-heria,WBTC (CTC) HABRA-DIGHA via Bally - 26|03:45,Non AC Seater (2+3),03:45,03h 20m,07:05,4.2,INR 130,37 Seats available
645,Habra to Heria,https://www.redbus.in/bus-tickets/habra-to-heria,WBTC (CTC) HABRA-DIGHA via Bally - 26|04:15,Non AC Seater (2+3),04:15,03h 20m,07:35,4.5,INR 130,26 Seats available
646,Habra to Heria,https://www.redbus.in/bus-tickets/habra-to-heria,WBTC (CTC) HABRA-DIGHA via Bally - 26|05:15,Non AC Seater (2+3),05:15,03h 20m,08:35,3.8,INR 130,31 Seats available
647,Midnapore to Kolkata Airport,https://www.redbus.in/bus-tickets/midnapore-to...,WBTC (CTC) Habra-Midnapur - 194|15:40,Non AC Seater (2+3),15:40,03h 00m,18:40,3.8,INR 111,38 Seats available
